In [60]:
import pandas as pd
import altair as alt
from datetime import datetime 
import dill
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import numpy as np

In [5]:
d_all_EOD = dill.load(open('data/d_all_EOD.pkd', 'rb'))
for ticker in d_all_EOD.keys():
    d_all_EOD[ticker] = d_all_EOD[ticker].reset_index(drop = True)

In [37]:
def adjust_prev_close(df,pred_type):
    last_close = df.iloc[-1]['close']
    prev_close = df['close'].shift(fill_value=0).values[1:]
    column = df[pred_type].values[1:]
    result = (column - prev_close)/prev_close
    
    transform = StandardScaler().fit(result.reshape(-1,1))
    values = transform.transform(result.reshape(-1,1))#[:,0]
    df = pd.DataFrame(values)
    df = df.rename(columns={0:pred_type})
    return df

In [71]:
def create_hist(df,title,col):
    hist = alt.Chart(df,height=150,width=200,title=title).mark_bar().encode(
        alt.X(col,bin=alt.Bin(maxbins=1000)),y=alt.Y('count()'))
    return hist
    mu,std=norm.fit(df.values)
    x = np.linspace(df.values.min(),df.values.max(),len(df.values))
    pdf = norm.pdf(x, mu, std)
    normal_curve = pd.DataFrame({'x': x, 'pdf': pdf})
    pdf_axis = alt.Axis(title='PDF', orient='right', grid=False)

    return alt.layer(hist,alt.Chart(normal_curve).mark_line(color='red').encode(
        x='x',
        y='pdf').configure_axisY(
    title='PDF',
    axis=pdf_axis
))

In [86]:
def create_chart(ticker,start,col,title,title2):
    df = d_all_EOD[ticker][start:]
    raw = create_hist(df,title,col)
    df = adjust_prev_close(df,col)
    norm = create_hist(df,title2,col)
    return raw & norm

In [87]:
chart1 = create_chart('SPY',0,'low','Prices (S&P)','Returns (S&P)')
chart2 = create_chart('COST',0,'low','Prices (COST)','Returns (COST)')
chart3 = create_chart('COST',-250,'low','Prices (COST 1yr)','Returns (COST 1yr)')
chart1 | chart2 | chart3

alt.HConcatChart(...)

In [75]:
df = d_all_EOD['COST']
col ='low'
raw = create_hist(df,'Raw Data ',col)
df = adjust_prev_close(df,col)
norm = create_hist(df,'Normalized Data',col)
chart1 & chart2

alt.VConcatChart(...)

In [76]:
df = d_all_EOD['COST'].iloc[-251:]
col ='low'
chart1 = create_hist(df,'Raw Data ',col)
df = adjust_prev_close(df,col)
chart2 = create_hist(df,'Normalized Data',col)
chart1 & chart2

alt.VConcatChart(...)